# Tugas 1 Machine Learning

### Nama : Adriati Manuk Allo
### NIM  : 2209105018

## **DAFTAR ISI**
- [DATASET](#0)
  - [0.1 Import Library](#0-1)
  - [0.2 Membaca dataset dari file csv](#0-2)
  - [0.3 Memeriksa jumlah kolom dan baris dalam dataset](#0-3)
- [DATA CLEANING](#1)
  - [1.1 Memeriksa jumlah missing value dalam masing-masing kolom](#1-1)
  - [1.2 Memeriksa tipe data di setiap kolom dan jumlah baris di setiap kolom](#1-2)
  - [1.3 Memeriksa nilai duplikasi di setiap kolom](#1-3)
  - [1.4 Memeriksa jumlah outlier dimasing-masing kolom bertipe numerik](#1-4)
  - [1.5 Menangani outlier pada kolom numerik](#1-5)
- [FEATURE ENGINEERING](#2)
  - [2.1 Konversi tipe data kategorigal ke numerik dengan One Hot Encoding](#2-1)
- [DATA TRANSFORMATION](#3)
  - [3.1 Normalisasi fitur numerik](#3-1)
- [MODELING](#4)
  - [4.1 Split Data](#4-1)
  - [4.2 Membangun model random forest](#4-2)
  - [4.3 Evaluasi model random forest](#4-3)
  - [4.4 Visualisasi Hasil](#4-4)

<a name="0"></a>
## **DATASET**

<blockquote> 

<br>**Deskripsi**</br>
Dataset ini dibuat secara sintetis untuk meniru data cuaca untuk tugas klasifikasi. Kumpulan data ini mencakup berbagai fitur terkait cuaca dan mengkategorikan cuaca menjadi empat jenis: Rainy (hujan), Sunny (cerah), Cloudy (berawan), and Snowy (bersalju). Dataset ini dapat diakses melalui link berikut __[Weather Classification Dataset](https://www.kaggle.com/datasets/nikhil7280/weather-type-classification)__.

<br>**Variabel**</br>
- *Temperature* (numerik): Suhu dalam derajat Celsius, mulai dari dingin ekstrem hingga panas ekstrem.
- *Humidity* (numerik): Persentase kelembaban, termasuk nilai di atas 100% untuk memperkenalkan outlier.
- *Wind Speed* (numerik): Kecepatan angin dalam kilometer per jam, dengan rentang termasuk nilai yang sangat tinggi.
- *Precipitation* (%) (numerik): Persentase curah hujan, termasuk nilai outlier.
- *Cloud Cover* (kategori): Deskripsi tutupan awan.
- *Atmospheric Pressure* (numerik): Tekanan atmosfer dalam hPa, yang mencakup rentang yang luas.
- *UV Index* (numerik): Indeks UV, yang menunjukkan kekuatan radiasi ultraviolet.
- *Season* (kategoris): Musim saat data direkam.
- *Visibility* (km) (numerik): Jarak pandang dalam kilometer, termasuk nilai yang sangat rendah atau sangat tinggi.
- *Location* (ketegori): Jenis lokasi tempat data direkam.
- *Weather Type* (ketegori): Variabel target untuk klasifikasi, yang menunjukkan jenis cuaca.

</blockquote>

>

<a name="0-1"></a>
## Import Library

<blockquote> Library Python yang digunakan antara lain: 
    
- Pandas
digunakan untuk memproses dan menganalisis data dalam bentuk tabel (DataFrame), seperti membaca file, manipulasi kolom/baris, dan cleaning data.
- Matplotlib
digunakan untuk membuat visualisasi data seperti grafik atau plot yang sederhana. Biasanya dipakai untuk eksplorasi data secara visual.
- Seaborn
merupakan library visualisasi berbasis Matplotlib yang menyediakan tampilan grafik yang lebih informatif dan estetis, sering digunakan untuk plot statistik seperti heatmap dan boxplot.
- OrdinalEncoder (dari sklearn.preprocessing)
digunakan untuk mengubah fitur kategorikal menjadi nilai numerik berdasarkan urutan. Cocok untuk fitur dengan urutan logis (ordinal).
- MinMaxScaler (dari sklearn.preprocessing)
digunakan untuk melakukan normalisasi fitur numerik ke dalam rentang 0 hingga 1 agar skala antar fitur sebanding.
- RandomForestClassifier (dari sklearn.ensemble)
digunakan untuk membangun model klasifikasi berbasis algoritma Random Forest, yaitu gabungan beberapa decision tree untuk meningkatkan akurasi.
- accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay (dari sklearn.metrics)
digunakan untuk mengevaluasi performa model klasifikasi dengan berbagai metrik evaluasi seperti akurasi, presisi, recall, dan f1-score, serta menampilkan confusion matrix.
- train_test_split, RandomizedSearchCV (dari sklearn.model_selection)
`train_test_split` digunakan untuk membagi dataset menjadi data latih dan data uji, sedangkan `RandomizedSearchCV` digunakan untuk melakukan hyperparameter tuning pada model.
- randint (dari scipy.stats)
digunakan untuk menghasilkan nilai integer acak, biasanya dipakai saat menentukan ruang pencarian (search space) untuk hyperparameter tuning.
</blockquote>

In [ ]:
# Data Preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

<a name="0-2"></a>
### Membaca dataset dari file csv

In [ ]:
### read dataset
df_weather = pd.read_csv('weather_classification_data.csv')
df_weather.head()

<a name="0-3"></a>
### Memeriksa jumlah kolom dan baris dalam dataset

In [ ]:
df_weather.shape

<a name="1"></a>
## **DATA CLEANING**

<a name="1-1"></a>
### Memeriksa jumlah missing value dalam masing-masing kolom

In [ ]:
df_weather.isnull().sum()

<a name="1-2"></a>
### Memeriksa tipe data di setiap kolom dan jumlah baris di setiap kolom

In [ ]:
df_weather.info()

<a name="1-3"></a>
### Memeriksa nilai duplikasi di setiap kolom

In [ ]:
df_weather.duplicated().sum()

<a name="1-4"></a>
### Memeriksa jumlah outlier dimasing-masing kolom bertipe numerik

In [ ]:
df_weather.describe()

In [ ]:
# ====== 1. Outlier Berdasarkan Z-Score ======
# Daftar kolom numerik yang akan dicek
columns_to_check = ['Temperature', 'Wind Speed', 
                    'Atmospheric Pressure', 'Visibility (km)', 'UV Index']

# Simpan jumlah outlier berdasarkan Z-Score
outliers_zscore = {}

for col in columns_to_check:
    z_scores = zscore(df_weather[col])
    outliers = df_weather[abs(z_scores) > 3]
    outliers_zscore[col] = outliers.shape[0]

print("Outlier Berdasarkan Z-Score (Sebelum Diubah):")
for col, count in outliers_zscore.items():
    print(f"- {col}: {count} outlier")

In [ ]:
# Set ukuran plot
plt.figure(figsize=(12, 6))

# Buat boxplot untuk masing-masing kolom numerik
sns.boxplot(data=df_weather[columns_to_check], orient="h", palette="Set2")

# Tambahkan judul dan label
plt.title("Outlier Berdasarkan Z-Score (Sebelum Penanganan Outliers)")
plt.xlabel("Nilai")
plt.ylabel("Fitur")

plt.tight_layout()
plt.show()

In [ ]:
# ====== 2. Outlier Berdasarkan Batas Logis ======
# Humidity dan Precipitation seharusnya tidak > 100
columns_to_check_2 = ['Humidity', 'Precipitation (%)']
# Membuat dictionary untuk menyimpan jumlah outlier
logical_outliers = {}

# Mengiterasi setiap kolom dalam columns_to_check untuk mengecek outlier
for column in columns_to_check_2:
    logical_outliers[f'{column} > 100'] = df_weather[df_weather[column] > 100].shape[0]

# Menampilkan jumlah outlier berdasarkan batas logis
print("\nOutlier Berdasarkan Batas Logis:")
for column, outlier_count in logical_outliers.items():
    print(f"- {column}: {outlier_count} outlier")

In [ ]:
# Set ukuran plot
plt.figure(figsize=(12, 6))

# Buat boxplot normal
sns.boxplot(data=df_weather[columns_to_check_2], orient="h", palette="Set2", showfliers=True)

# Tambahkan titik merah untuk nilai > 100
for i, col in enumerate(columns_to_check_2):
    outliers = df_weather[df_weather[columns_to_check_2] > 100][col]
    plt.scatter(outliers, [i]*len(outliers), color='grey', label='Outlier > 100' if i == 0 else "", zorder=10)

# Label dan title
plt.title("Visualisasi Outlier Berdasarkan Batas Logis (>100)")
plt.xlabel("Nilai")
plt.ylabel("Fitur")
plt.legend()
plt.tight_layout()
plt.show()

<blockquote> Berdasarkan hasil analisis, outlier pada kolom humidity dan precipitation perlu ditangani karena nilainya tidak masuk akal, seperti melebihi 100%. Sementara itu, meskipun outlier pada kolom temperature, wind speed, atmospheric pressure, dan visibility masih mungkin terjadi secara alami, saya memutuskan untuk menangani semua outlier demi menjaga kualitas data dan menghindari gangguan pada hasil model.
</blockquote> 

<a name="1-5"></a>
### Menangani outlier pada kolom numerik

In [ ]:
# Salin data asli
df_cleaned = df_weather.copy()

In [ ]:
### Imputation outliers dengan nilai median pada kolom Temperature, Wind Speed, Atmospheric Pressure', 'Visibility (km)
# Loop hingga tidak ada lagi outlier
while True:
    # Hitung Z-score untuk setiap kolom
    z_scores_df = df_cleaned[columns_to_check].apply(zscore)
    
    # Buat mask: True untuk baris yang tidak mengandung outlier di semua kolom
    mask = (z_scores_df.abs() <= 3).all(axis=1)
    
    # Jika tidak ada baris yang dihapus (sudah bersih), hentikan loop
    if mask.sum() == df_cleaned.shape[0]:
        break
    
    # Update dataframe hanya dengan baris yang aman
    df_cleaned = df_cleaned[mask]

# Cek jumlah outlier akhir
print("Outlier Berdasarkan Z-Score (Setelah Penghapusan Iteratif):")
for col in columns_to_check:
    z_scores = zscore(df_cleaned[col])
    count = (abs(z_scores) > 3).sum()
    print(f"- {col}: {count} outlier")

In [ ]:
# Visualisasi setelah penanganan
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_cleaned[columns_to_check], orient="h", palette="Set3")
plt.title("Box Plot Setelah Penanganan Outlier (Z-Score)")
plt.xlabel("Nilai")
plt.ylabel("Fitur")
plt.tight_layout()
plt.show()

In [ ]:
### Drop outliers pada kolom humidity dan precipitation
df_cleaned = df_cleaned[(df_cleaned['Humidity'] <= 100) & (df_cleaned['Precipitation (%)'] <= 100)]

# Humidity dan Precipitation seharusnya tidak > 100
columns_to_check_2 = ['Humidity', 'Precipitation (%)']

# Membuat dictionary untuk menyimpan jumlah outlier
logical_outliers = {}

# Mengiterasi setiap kolom dalam columns_to_check untuk mengecek outlier
for column in columns_to_check_2:
    logical_outliers[f'{column} > 100'] = df_cleaned[df_cleaned[column] > 100].shape[0]

# Menampilkan jumlah outlier berdasarkan batas logis
print("\nOutlier Berdasarkan Batas Logis:")
for column, outlier_count in logical_outliers.items():
    print(f"- {column}: {outlier_count} outlier")

In [ ]:
# Set ukuran plot
plt.figure(figsize=(12, 6))

# Buat boxplot normal
sns.boxplot(data=df_cleaned[columns_to_check_2], orient="h", palette="Set2", showfliers=True)

# Tambahkan titik merah untuk nilai > 100
for i, col in enumerate(columns_to_check_2):
    outliers = df_cleaned[df_cleaned[col] > 100][col]  # Perbaikan di sini
    plt.scatter(outliers, [i]*len(outliers), color='grey', label='Outlier > 100' if i == 0 else "", zorder=10)

# Label dan title
plt.title("Visualisasi Outlier Berdasarkan Batas Logis (>100)")
plt.xlabel("Nilai")
plt.ylabel("Fitur")
plt.legend()
plt.tight_layout()
plt.show()

<blockquote>Untuk menangani outliers pada kolom temperature, humidity, wind speed, precipitation, Atmospheric Pressure', 'Visibility (km), dan UV Index dilakukan penghapusan pada baris yang memiliki outliers. Pada kolom humidity dan precipitation, outlier di cek menggunakan IQT, sedangkan  kolom temperature, wind speed, atmospheric pressure', 'visibility, dan UV Index menggunakan Z-Score. Hal ini dilakukan karena jumlah outliers yang ada pada  kolom-kolom ini sedikit, sehingga ketika di hapus tidak memberikan pengaruh terhadap akurasi model.</blockquote>

In [ ]:
df_cleaned.describe()

<a name="2"></a>
## **FEATURE ENGINEERING**

<a name="2-1"></a>
### Konversi tipe data kategorigal ke numerik dengan One Hot Encoding

In [ ]:
df_cleaned['Cloud Cover'].unique()

In [ ]:
df_cleaned['Season'].unique()

In [ ]:
df_cleaned['Location'].unique()

In [ ]:
eo = OrdinalEncoder()
df_cleaned[['Cloud Cover']] = eo.fit_transform(df_cleaned[['Cloud Cover']])
df_cleaned[['Season']] = eo.fit_transform(df_cleaned[['Season']])
df_cleaned[['Location']] = eo.fit_transform(df_cleaned[['Location']])
df_cleaned

<a name="3"></a>
## **DATA TRANSFORMATION**

<a name="3-1"></a>
### Normalisasi fitur numerik

In [ ]:
numeric_columns = ['Temperature', 'Humidity', 'Wind Speed', 
                     'Precipitation (%)', 'Atmospheric Pressure', 
                     'UV Index', 'Visibility (km)']
sc = StandardScaler()
normalized_data = sc.fit_transform(df_cleaned[numeric_columns])

df_cleaned_normalized = df_cleaned.copy()
df_cleaned_normalized[numeric_columns] = normalized_data
df_cleaned_normalized.head()

<blockquote>Normalisasi dilakukan pada kolom temperature, humidity, wind speed, dan precipitation agar tidak ada fitur yang mendominasi karena skala yang besar. Selain itu, karena sebelumnya telah dilakukan penghapusan outlier, maka penggunaan MinMaxScaler dirasa lebih baik dibandingkan StandardScaler</blockquote>

<a name="4"></a>
## **MODELING**

<blockquote> 
Algoritma Random Forest dipilih untuk tugas klasifikasi cuaca karena kemampuannya menangani data numerik dan kategoris, serta cocok untuk klasifikasi multikelas seperti Rainy, Sunny, Cloudy, dan Snowy. Random Forest bekerja dengan membangun banyak pohon keputusan dari subset data dan fitur secara acak, lalu menggabungkan hasil prediksinya melalui voting mayoritas.

<br>Metode ini unggul karena:</br>
- Robust terhadap outlier dan noise, yang penting karena dataset mengandung nilai ekstrem.
- Tidak mudah overfitting, berkat pendekatan ensemble-nya.
- Dapat memberikan feature importance, membantu memahami fitur paling berpengaruh terhadap prediksi cuaca.
</blockquote> 


<a name="4-1"></a>
### Split Data

In [ ]:
### Split data menjadi features (X) dan target (y)
X = df_cleaned_normalized.drop('Weather Type', axis=1)
y = df_cleaned_normalized['Weather Type']

### Split data menjadi data training dan data testing (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<a name="4-2"></a>
### Membangun model random forest

In [ ]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

rf = RandomForestClassifier()
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)
rand_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

<a name="4-3"></a>
### Evaluasi model random forest

In [ ]:
y_pred = best_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print ("Accuracy;", accuracy)
print ("Precision;", precision)
print ("Recall;", recall )
print ("F1 Score;", f1)

<a name="4-4"></a>
### Visualisasi Hasil 

In [ ]:
ConfusionMatrixDisplay(confusion_matrix=cm).plot();